# Project Assignment : Attribution Data

This notebook is an assignment from Codecademy Data Science Path - Section 5: Analyze Real Data with SQL.

CoolTShirts, an innovative apparel shop, is running a bunch of marketing campaigns. In this project, I’ll be helping them answer these questions about their campaigns.

=========================================================================================================================

Import related sqlalchemy module and setup database connection

In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///attribution.db')
connection = engine.connect()

In [3]:
# setting the SQL Magic Commands
%load_ext sql
%sql sqlite:///attribution.db

'Connected: @attribution.db'

Print tables in attribution.db

In [4]:
print(engine.table_names())

['page_visits']


First, lets get an overview of the data:

In [5]:
%%sql

SELECT *
FROM page_visits
LIMIT 15;

 * sqlite:///attribution.db
Done.


page_name,timestamp,user_id,utm_campaign,utm_source
1 - landing_page,2018-01-24 03:12:16,10006,getting-to-know-cool-tshirts,nytimes
2 - shopping_cart,2018-01-24 04:04:16,10006,getting-to-know-cool-tshirts,nytimes
3 - checkout,2018-01-25 23:10:16,10006,weekly-newsletter,email
1 - landing_page,2018-01-25 20:32:02,10030,ten-crazy-cool-tshirts-facts,buzzfeed
2 - shopping_cart,2018-01-25 23:05:02,10030,ten-crazy-cool-tshirts-facts,buzzfeed
3 - checkout,2018-01-28 13:26:02,10030,retargetting-campaign,email
4 - purchase,2018-01-28 13:38:02,10030,retargetting-campaign,email
1 - landing_page,2018-01-05 18:31:17,10045,getting-to-know-cool-tshirts,nytimes
2 - shopping_cart,2018-01-05 21:16:17,10045,getting-to-know-cool-tshirts,nytimes
3 - checkout,2018-01-09 03:05:17,10045,retargetting-ad,facebook


In [6]:
%%sql

SELECT COUNT(*)
FROM page_visits;

 * sqlite:///attribution.db
Done.


COUNT(*)
5692


There are 5692 row(s) in this data.

## 1. Getting familiar with the company

### 1.1. How many campaigns and sources does CoolTShirts use and how are they related?

Campaigns that CoolTShirts is running is as follows.

In [7]:
%%sql

/*Campaigns*/
SELECT DISTINCT(utm_campaign)
FROM page_visits;

 * sqlite:///attribution.db
Done.


utm_campaign
getting-to-know-cool-tshirts
weekly-newsletter
ten-crazy-cool-tshirts-facts
retargetting-campaign
retargetting-ad
interview-with-cool-tshirts-founder
paid-search
cool-tshirts-search


Campaign sources of CoolTShirts company are:

In [8]:
%%sql

/*Sources*/
SELECT DISTINCT(utm_source)
FROM page_visits;

 * sqlite:///attribution.db
Done.


utm_source
nytimes
email
buzzfeed
facebook
medium
google


We can see the relationship between campaign and source in this table below

In [9]:
%%sql

/*Campaigns & Sources Relationship*/
SELECT DISTINCT utm_campaign, utm_source
FROM page_visits;

 * sqlite:///attribution.db
Done.


utm_campaign,utm_source
getting-to-know-cool-tshirts,nytimes
weekly-newsletter,email
ten-crazy-cool-tshirts-facts,buzzfeed
retargetting-campaign,email
retargetting-ad,facebook
interview-with-cool-tshirts-founder,medium
paid-search,google
cool-tshirts-search,google


- Campaign: Identifies the specific ad or email blast (i.e., june-21- newsletter or memorial-day-sale)
- Source: Identifies which site sent the traffic (i.e., google, newsletter, or facebook_ad)

1 utm_campaigns correspond to 1 source, but 1 source can have more than 1 campaign.

### 1.2. What pages are on the website?

In [10]:
%%sql

/*Pages on Site*/
SELECT DISTINCT(page_name)
FROM page_visits;

 * sqlite:///attribution.db
Done.


page_name
1 - landing_page
2 - shopping_cart
3 - checkout
4 - purchase


There are 4 pages on CoolTShirts website, which is: landing_page, shopping_cart, checkout, and purchase. These pages resembles a user journey to purchase an item from their website.

## 2. User Journey

### 2.1. How many first touches is each campaign responsible for?

In [11]:
%%sql

/*First Touch*/
WITH first_touch AS (
    SELECT user_id,
        MIN(timestamp) as first_touch_at
    FROM page_visits
    GROUP BY user_id),
    ft_attr AS( 
    SELECT ft.user_id,
               ft.first_touch_at,
               pv.utm_source,
               pv.utm_campaign
    FROM first_touch as ft
      JOIN page_visits as PV
        ON ft.user_id = pv.user_id
        AND ft.first_touch_at = pv.timestamp
    )
SELECT ft_attr.utm_campaign,
        ft_attr.utm_source,
        COUNT(*)
FROM fT_attr
GROUP BY 1,2
ORDER BY 3 DESC;

 * sqlite:///attribution.db
Done.


utm_campaign,utm_source,COUNT(*)
interview-with-cool-tshirts-founder,medium,622
getting-to-know-cool-tshirts,nytimes,612
ten-crazy-cool-tshirts-facts,buzzfeed,576
cool-tshirts-search,google,169


### 2.2. How many last touches is each campaign responsible for?

In [15]:
%%sql

WITH last_touch AS (
    SELECT user_id,
        MAX(timestamp) as last_touch_at
    FROM page_visits
    GROUP BY user_id),
    lt_attr AS( 
    SELECT lt.user_id,
               lt.last_touch_at,
               pv.utm_source,
               pv.utm_campaign
    FROM last_touch as lt
      JOIN page_visits as PV
        ON lt.user_id = pv.user_id
        AND lt.last_touch_at = pv.timestamp
    )
SELECT lt_attr.utm_campaign,
        lt_attr.utm_source,
        COUNT(*)
FROM lt_attr
GROUP BY 1,2
ORDER BY 3 DESC;

 * sqlite:///attribution.db
Done.


utm_campaign,utm_source,COUNT(*)
weekly-newsletter,email,447
retargetting-ad,facebook,443
retargetting-campaign,email,245
getting-to-know-cool-tshirts,nytimes,232
ten-crazy-cool-tshirts-facts,buzzfeed,190
interview-with-cool-tshirts-founder,medium,184
paid-search,google,178
cool-tshirts-search,google,60


### 2.3. How many visitors make a purchase?

In [13]:
%%sql

SELECT COUNT(DISTINCT(user_id))
FROM page_visits
WHERE page_name = '4 - purchase';

 * sqlite:///attribution.db
Done.


COUNT(DISTINCT(user_id))
361


There are 361 different visitors that make a purchase on CoolTShirts site

### 2.4. How many last touches on the purchase page is each campaign responsible for?

In [14]:
%%sql

WITH last_touch AS (
    SELECT user_id,
        MAX(timestamp) as last_touch_at
    FROM page_visits
    WHERE page_name = '4 - purchase'
    GROUP BY user_id),
    lt_attr AS( 
    SELECT lt.user_id,
               lt.last_touch_at,
               pv.utm_source,
               pv.utm_campaign,
              pv.page_name
    FROM last_touch as lt
      JOIN page_visits as PV
        ON lt.user_id = pv.user_id
        AND lt.last_touch_at = pv.timestamp
    )
SELECT lt_attr.utm_campaign,
        lt_attr.utm_source,
        COUNT(*)
FROM lt_attr
GROUP BY 1,2
ORDER BY 3 DESC;

 * sqlite:///attribution.db
Done.


utm_campaign,utm_source,COUNT(*)
weekly-newsletter,email,115
retargetting-ad,facebook,113
retargetting-campaign,email,54
paid-search,google,52
getting-to-know-cool-tshirts,nytimes,9
ten-crazy-cool-tshirts-facts,buzzfeed,9
interview-with-cool-tshirts-founder,medium,7
cool-tshirts-search,google,2


### 2.5. What is the typical user journey?

In [16]:
%%sql

SELECT page_name,COUNT(*)
FROM page_visits
GROUP BY page_name;

 * sqlite:///attribution.db
Done.


page_name,COUNT(*)
1 - landing_page,2000
2 - shopping_cart,1900
3 - checkout,1431
4 - purchase,361


These are the completion rate for each page, as a ratio to previous page:
- landing_page : 2000
- shopping_cart : 1900 (95%)
- checkout : 1431 (75,32%)
- purchase : 361 (25,23%)

only 1 in 4 users purchased items after they completed their checkout!

## 3. Optimizing the Campaign Budget

### 3.1. CoolTShirts can re-invest in 5 campaigns. Which should they pick and why?

CoolTShirts should focus on 5 existing campaigns that resulted in more purchase, as shown in the table below

In [17]:
%%sql

WITH last_touch AS (
    SELECT user_id,
        MAX(timestamp) as last_touch_at
    FROM page_visits
    WHERE page_name = '4 - purchase'
    GROUP BY user_id),
    lt_attr AS( 
    SELECT lt.user_id,
               lt.last_touch_at,
               pv.utm_source,
               pv.utm_campaign,
              pv.page_name
    FROM last_touch as lt
      JOIN page_visits as PV
        ON lt.user_id = pv.user_id
        AND lt.last_touch_at = pv.timestamp
    )
SELECT lt_attr.utm_campaign,
        lt_attr.utm_source,
        COUNT(*)
FROM lt_attr
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 5;

 * sqlite:///attribution.db
Done.


utm_campaign,utm_source,COUNT(*)
weekly-newsletter,email,115
retargetting-ad,facebook,113
retargetting-campaign,email,54
paid-search,google,52
getting-to-know-cool-tshirts,nytimes,9


## 4. Conclusion

Based on section 3.1, CoolTShirts should focus on these campaigns:
- weekly-newsletter (115 purchases)
- retargetting-ad (113 purchases)
- retargetting-campaign	(54 purchases)
- paid-search (52 purchases)
- getting-to-know-cool-tshirts (9 purchases)